## A Guide to Digitizing Historical Maps Using QGIS, Python, and Lightroom

Authors: Haicheng Xu & Idaliya Grigoryeva

### Intro

This guide will walk you through the complete process of digitizing a historical map, using a map of Indonesia from our project as an example. Our goal is to digitize the road network in Indonesia to create a friction surface that represents the travel cost between any two points. With this objective in mind, we focused on finding a map where the roads are clearly distinguishable from other features. To achieve accurate digitization, we selected a map where only the roads are red, making them easy to extract.

Steps for digitizing: 
1. Eliminate the saturation for all colors except for red in Lightroom
2. Use map_to_bw function below to convert the processed map from step 1 to black and white
3. Import the black and white map from step 2 into QGIS
4. Convert the raster to points then connect the points using nearest points to path in QGIS


### Map Download

You can access the Indonesia map here: [Indonesia Map](https://digitalcollections.universiteitleiden.nl/view/item/814037?solr_nav%5Bid%5D=425f4cdcd85c99e9fcfd&solr_nav%5Bpage%5D=0&solr_nav%5Boffset%5D=11).

Note: Please download the "Original Master" version of the map for the best result.

### Lightroom

1. Eliminate the saturation for all colors except for red.

2. Boost overall saturation to 100 to make it easier for python program (next step) to identify the red.

    <img src="Images/lr-1.jpg" alt="Saturation adjustments" width="500"/>



3. Decrease the saturation of midtone and shadow areas to remove some red noise from the yellow background of the map.

4. Increase the saturation of highlight areas to make the red more prominent.

    <img src="Images/lr-2.jpg" alt="Color grading adjustments" width="500"/>


5. Using color noise reduction to further reduce the red noise due to the yellow background.

    <img src="Images/lr-3.jpg" alt="Image Description" width="500"/>


6. Export the image from lightroom and select large size. 

    <img src = 'Images/lr-5.jpg' alt = 'Lr export' width="500">

### map_to_bw processing using Python

In [3]:
from PIL import Image

In [7]:
def map_to_bw (image_path):
    # Open the image
    im = Image.open(image_path)

    # Convert the image to RGB mode if it's not in that mode already
    im = im.convert("RGB")

    # Get the pixel access object
    pix = im.load()

    # Iterate through each pixel and keep only red pixels
    for x in range(im.width):
        for y in range(im.height):
            r, g, b = pix[x, y]

            if (r-g + r-b) > 80:
                # set red pixels
                pix[x, y] = (256, 256, 256)
            else:
                # Set colorless pixels to black
                pix[x, y] = (0, 0, 0)

    # Save or display the modified image
    im.save('Images/Sulawesi_bw.png')

In [8]:
map_to_bw('Images/Sulawesi_processed.jpg')

/Users/haichengxu/anaconda3/lib/python3.11/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (107641488 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


### QGIS Digitization

1. Download QuickMapService plugin if you don’t have it 
2. Open a OSM standard layer

    <img src = "Images/osm_layer.jpg"  alt="opening osm layer" width="500"/>


3. Georeference the Sulawesi_bw.png onto OSM standard using this method: https://www.youtube.com/watch?v=jKLBFddpTGI

    Note: add 10-20 reference points throughout the North, South, East, and West corners of the map for the best result

4. Click into layer property by double clicking the Sulawesi_bw layer and change its transparency to 50%.

5. Resulting Georeferenced map of Sulawesi

    <img src = "Images/georeferenced_result-1.jpg"  alt="opening osm layer" width="500"/>

    <img src = "Images/georeferenced_result-2.jpg"  alt="opening osm layer" width="500"/>

    <img src = "Images/georeferenced_result-3.jpg"  alt="opening osm layer" width="500"/>



6. Using raster pixels to points function to convert roads in white to vector/point data

7. Run the function with the modified map as input




8. Then right click on vector points → properties →  Symbology → categorized (from the top drop-down menu → select value (from the value drop-down menu) → classify

9. After classifying, go to sources (we’re still in properties menu) → provide feature filter → query builder → type "VALUE" = 255 → OK